In [2]:
%pip install tensorflow

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [3]:
%pip install opencv-python numpy tensorflow


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [4]:
import cv2
import numpy as np
from tensorflow.keras.models import load_model
from collections import Counter
import time
import webbrowser

In [5]:
model = load_model("emotion_model.h5")
emotions = ['Angry', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprise', 'Neutral']

In [ ]:
import cv2
import numpy as np
import time

# Load model and emotions list before this (assumed already defined)
# model = ...
# emotions = [...]

# Start webcam
cap = cv2.VideoCapture(0)
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")

detected_moods = []
start_time = time.time()
duration = 60  # seconds

locked_face_coords = None
last_seen_time = None
face_lost_threshold = 20  # seconds


def is_face_still_present(frame_gray, locked_coords):
    """Check if a face still exists in the approximate area."""
    x, y, w, h = locked_coords
    search_area = frame_gray[max(y-20, 0):y+h+20, max(x-20, 0):x+w+20]
    faces = face_cascade.detectMultiScale(search_area, 1.3, 5)
    return len(faces) > 0


while time.time() - start_time < duration:
    ret, frame = cap.read()
    if not ret:
        break

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    current_time = time.time()

    if locked_face_coords is None:
        faces = face_cascade.detectMultiScale(gray, 1.3, 5)
        if len(faces) > 0:
            locked_face_coords = faces[0]
            last_seen_time = current_time
    else:
        if is_face_still_present(gray, locked_face_coords):
            last_seen_time = current_time
            x, y, w, h = locked_face_coords
            roi_gray = gray[y:y+h, x:x+w]

            try:
                roi_resized = cv2.resize(roi_gray, (48, 48))
                roi_input = roi_resized.reshape(1, 48, 48, 1) / 255.0
                pred = model.predict(roi_input)
                confidence = np.max(pred)

                if confidence > 0.6:
                    mood = emotions[np.argmax(pred)]
                    detected_moods.append(mood)
                    cv2.putText(frame, f"{mood} ({confidence:.2f})",
                                (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX,
                                1, (255, 255, 255), 2)
                    cv2.rectangle(frame, (x, y), (x + w, y + h),
                                  (255, 255, 255), 2)

            except Exception as e:
                print("Error processing face ROI:", e)
        else:
            # Face not seen
            if current_time - last_seen_time > face_lost_threshold:
                print("Face lost for over 20 seconds. Reacquiring...")
                locked_face_coords = None
                last_seen_time = None

    cv2.imshow("Webcam Mood Detection", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


In [16]:
#Most common mood
#print("All detected moods:", detected_moods)

final_mood = Counter(detected_moods).most_common(1)[0][0]
print(f"Dominant Mood Over {duration} Seconds: {final_mood}")

IndexError: list index out of range

In [7]:
from collections import Counter
if detected_moods:
    final_mood = Counter(detected_moods).most_common(1)[0][0]
    print(f"Dominant Mood Over {duration} Seconds: {final_mood}")

    with open("mood.txt", "w") as f:
        f.write(final_mood)


Dominant Mood Over 60 Seconds: Happy


In [8]:
# Map moods to YouTube playlists (customize URLs as you like)
mood_to_youtube = {
    "Happy": "https://www.youtube.com/watch?v=ZbZSe6N_BXs&list=PLFgquLnL59alCl_2TQvOiD5Vgm1hCaGSI",
    "Sad": "https://www.youtube.com/watch?v=Ho32Oh6b4jc&list=PLDcnymzs18LVXfO_x0Ei0R24qDbVtyy66",
    "Angry": "https://www.youtube.com/watch?v=HgzGwKwLmgM",
    "Neutral": "https://www.youtube.com/watch?v=5qap5aO4i9A",
    "Surprise": "https://www.youtube.com/watch?v=2Vv-BfVoq4g",
    "Fear": "https://www.youtube.com/watch?v=CevxZvSJLk8",
    "Disgust": "https://www.youtube.com/watch?v=JGwWNGJdvx8",
}

# Open YouTube playlist for detected mood
url = mood_to_youtube.get(final_mood)
if url:
    print(f"Opening YouTube playlist for mood: {final_mood}")
    webbrowser.open(url)
else:
    print(f"No YouTube playlist found for mood: {final_mood}")

Opening YouTube playlist for mood: Happy


In [11]:
import serial
import time
import os

# Arduino serial configuration
arduino_port = 'COM7'  # Update as needed
baud_rate = 9600
mood_file = 'mood.txt'

try:
    # Initialize serial connection
    ser = serial.Serial(arduino_port, baud_rate, timeout=1)
    time.sleep(2)  # Give Arduino time to reset

    # ✅ Check if mood.txt exists
    if os.path.exists(mood_file):
        with open(mood_file, 'r') as file:
            final_mood = file.read().strip().lower()

        if final_mood:
            # ✅ Send mood to Arduino
            ser.write((final_mood + '\n').encode())
            print(f"✅ Sent mood '{final_mood}' to Arduino")
        else:
            print("⚠️ Mood file is empty. No mood to send.")
    else:
        print("❌ mood.txt not found. Please generate it with your detection script.")

    ser.close()

except Exception as e:
    print(f"❌ Error: {e}")


✅ Sent mood 'happy' to Arduino


In [8]:
%pip install SpeechRecognition pyttsx3 pyaudio openai-whisper youtube-search-python pytube

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [8]:
%pip install pywhatkit

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import speech_recognition as sr
import pyttsx3
import pywhatkit
import serial
import time
import os
import sys

# Initialize voice engine
engine = pyttsx3.init()
engine.setProperty('rate', 160)

# Configure Arduino
arduino_port = 'COM8'  # Change as needed
baud_rate = 9600

try:
    ser = serial.Serial(arduino_port, baud_rate, timeout=1)
    time.sleep(2)
    print("✅ Connected to Arduino")
except Exception as e:
    print(f"❌ Arduino not connected: {e}")
    ser = None

# Speak safely
def speak(text):
    print("Aura:", text)
    try:
        engine.say(text)
        engine.runAndWait()
    except RuntimeError:
        pass

# Listen to voice
def listen_for_command():
    r = sr.Recognizer()
    with sr.Microphone() as source:
        print("🎤 Listening...")
        audio = r.listen(source, phrase_time_limit=5)
    try:
        command = r.recognize_google(audio)
        print("You said:", command)
        return command.lower()
    except sr.UnknownValueError:
        print("❗ Sorry, I didn't catch that.")
        return ""
    except sr.RequestError:
        print("❗ Speech recognition error.")
        return ""

# Send data to Arduino
def send_to_arduino(msg):
    if ser:
        ser.write((msg.strip().lower() + '\n').encode())
        print(f"📤 Sent to Arduino: {msg}")

# Play YouTube
def play_youtube(query):
    speak(f"Playing {query} ambience")
    pywhatkit.playonyt(query + " ambience")

# Process command
def process_command(cmd):
    if "stop aura" in cmd or "shutdown" in cmd:
        send_to_arduino("off")
        speak("Aura assistant shutting down. Goodbye!")
        if ser:
            ser.close()
        sys.exit()

    elif "turn off ambience" in cmd or "turn off lights" in cmd:
        send_to_arduino("off")
        speak("Turning off the ambience lights.")

    elif "turn off music" in cmd or "stop music" in cmd:
        speak("Please close the YouTube tab to stop music.")  # Can't close browser automatically

    elif "happy" in cmd:
        send_to_arduino("happy")
        play_youtube("happy music")

    elif "sad" in cmd:
        send_to_arduino("sad")
        play_youtube("sad music")

    elif "angry" in cmd:
        send_to_arduino("angry")
        play_youtube("angry music")

    elif "relax" in cmd or "neutral" in cmd:
        send_to_arduino("neutral")
        play_youtube("relaxing ambience")

    elif "library" in cmd:
        send_to_arduino("library")
        play_youtube("library ambience with bright light")

    elif "cafe" in cmd:
        send_to_arduino("cafe")
        play_youtube("cafe ambience")

    elif "fear" in cmd:
        send_to_arduino("fear")
        play_youtube("deep ocean ambience")

    elif "surprise" in cmd:
        send_to_arduino("surprise")
        play_youtube("mystical forest ambience")

    else:
        speak("Sorry, I didn't understand that command.")

# Main loop
def aura_loop():
    print("👂 Say 'Hey Aura' to activate...")
    while True:
        command = listen_for_command()
        if "hey aura" in command:
            speak("Yes, I'm listening.")
            user_cmd = listen_for_command()
            process_command(user_cmd)

try:
    aura_loop()
except KeyboardInterrupt:
    if ser:
        ser.close()
    speak("Aura assistant stopped.")


❌ Arduino not connected: could not open port 'COM8': FileNotFoundError(2, 'The system cannot find the file specified.', None, 2)
👂 Say 'Hey Aura' to activate...
🎤 Listening...
You said: Ek Aur
🎤 Listening...
